---------------------
# **P2 - Player Classification** 🥏
---------------------

In [1]:
# IMPORTS
import pandas as pd
import seaborn as sns

In [2]:
# Color Palette
palette = {
    'ltBlu': '#8ecae6',
    'mdBlu': '#219ebc',
    'dkBlu': '#023047',
    'gold': '#ffb703',
    'orange': '#fb8500'
}

In [3]:
# Get raw data
df = pd.read_csv(r'.\DATA\raw\20220227_player_all_time.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2558 entries, 0 to 2557
Data columns (total 26 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Player  2558 non-null   object
 1   G       2558 non-null   int64 
 2   PP      2558 non-null   int64 
 3   POS     2558 non-null   int64 
 4   SCR     2558 non-null   int64 
 5   AST     2558 non-null   int64 
 6   GLS     2558 non-null   int64 
 7   BLK     2558 non-null   int64 
 8   +/- ▼   2558 non-null   int64 
 9   Cmp     2558 non-null   int64 
 10  Cmp%    2558 non-null   object
 11  Y       2558 non-null   int64 
 12  TY      2558 non-null   int64 
 13  RY      2558 non-null   int64 
 14  OEFF    2558 non-null   object
 15  HA      2558 non-null   int64 
 16  T       2558 non-null   int64 
 17  S       2558 non-null   int64 
 18  D       2558 non-null   int64 
 19  C       2558 non-null   int64 
 20  Hck     2558 non-null   int64 
 21  Hck%    2558 non-null   object
 22  Pul     2558 non-null   

# GAME PLAN
**DATA CLEANING**
1. clean up stat column names
1. Verify dtypes are correct (i.e. OEFF = object)
1. Assess null information
1. Assign positions to players
1. Any Data Engineering?

**EDA**
1. Single variable EDA, focus on ___?
1. Multi-variable EDA, focus on differences in position
1. Correlation heatmap
1. Create df where relevant values are by points played
    a. If any highly collinear after, remove
1. Remove one of any pairs of highly collinear variables
1. Create DF for just players w/ position

In [5]:
# Clean up colum names
col_names = {'Player': 'player',
             'G': 'games',
             'PP': 'points_played',
             'POS': 'off_possessions',
             'SCR': 'scores', # assists + goals
             'AST': 'assists',
             'GLS': 'goals',
             'BLK': 'blocks',
             '+/- ▼': 'plus_minus', # +1 per goal/assist/block, -1 per throwaway and drop
             'Cmp': 'completions',
             'Cmp%': 'completion_pct', # completion pct, must have >= 100 throw attempts
             'Y': 'total_yards',
             'TY': 'throwing_yards',
             'RY': 'receiving_yards',
             'OEFF': 'off_efficiency', # team scores while on field / off. possessions
             'HA': 'hockey_assists',
             'T': 'throwaways',
             'S': 'stalls',
             'D': 'drops',
             'C': 'callahans',
             'Hck': 'hucks', # > 40yd throws downfield
             'Hck%': 'huck_pct', # completion pct, must have 10+ huck attempts
             'Pul': 'pulls',
             'OPP': 'off_points_played',
             'DPP': 'def_points_played',
             'MP': 'minutes_played'
}

df.columns = [col_names[i] for i in df.columns]

In [6]:
# Sample
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2558 entries, 0 to 2557
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   player             2558 non-null   object
 1   games              2558 non-null   int64 
 2   points_played      2558 non-null   int64 
 3   off_possessions    2558 non-null   int64 
 4   scores             2558 non-null   int64 
 5   assists            2558 non-null   int64 
 6   goals              2558 non-null   int64 
 7   blocks             2558 non-null   int64 
 8   plus_minus         2558 non-null   int64 
 9   completions        2558 non-null   int64 
 10  completion_pct     2558 non-null   object
 11  total_yards        2558 non-null   int64 
 12  throwing_yards     2558 non-null   int64 
 13  receiving_yards    2558 non-null   int64 
 14  off_efficiency     2558 non-null   object
 15  hockey_assists     2558 non-null   int64 
 16  throwaways         2558 non-null   int64 


In [7]:
len(df[df['hucks'] == 0])

2144

In [8]:
# Transform Object Columns into int/float
df[['completion_pct', 'off_efficiency', 'huck_pct']]
df.replace(inplace=True, to_replace='--', value=None)

In [9]:
num_cols = ['completion_pct', 'off_efficiency', 'huck_pct']
for i in num_cols:
    df[i] = df[i].astype('float')

df.info give the following information: 
- completion_pct     1317 non-null   object
- off_efficiency     1574 non-null   object
- huck_pct           107 non-null    object

`completion_pct` is **48.5%** null </br>
`off_efficiency` is **38.5%** null </br>
`huck_pct` is **95.8%** null </br>

Because of the high presence of missing values in `huck_pct`, this column is dropped from the finall processed dataset. Similarly, `hucks` is also dropped because the large majority of values is 0. For `completion_pct` and `off_efficiency`, these will be left in along with null values. Many clustering algorithms 

In [10]:
df.drop(labels=['huck_pct', 'hucks'], axis=1, inplace=True)

In [11]:
# Assign Positions to Players
player_pos = pd.read_csv("DATA/raw/20220304_player_positions.csv").drop(labels='Unnamed: 0.1', axis=1)
player_pos.sample(5)

,Unnamed: 0,player_link,position
2312,Scott Hampton,https://theaudl.com/league/players/shampton,NaN
2548,Trevor Knechtel,https://theaudl.com/league/players/tknechtel,NaN
985,Grant Franke,https://theaudl.com/league/players/gfranke,NaN
1584,Kyle Dew,https://theaudl.com/league/players/kdew,Hybrid
1179,Jarnail Bajwa,https://theaudl.com/league/players/jbajwa,NaN


In [12]:
player_pos = player_pos.rename({'Unnamed: 0': 'player_name'}, axis=1)

In [13]:
player_pos.set_index('player_name', inplace=True)
merged = df.join(other=player_pos, on='player', how='left')

In [18]:
# Drop link col and save
# merged.drop(labels='player_link', axis=1, inplace=True)
merged[~pd.isna(merged['position'])].sample(5)

# SAVE FILE
# merged.to_csv(".\\DATA\\postproc\\players.csv", index=False)